# [Projects] 신용거래 이상탐지 데이터

카드 거래 이력이 담긴 fraud.csv 데이터를 기반으로

신용카드 이상감지 모델을 만들기 위해 데이터를 정제한다.  
해당 데이터셋에서는 `is_fraud`가 사기거래 여부를 나타내는 종속변수로 작용한다.

## 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
cc_df = pd.read_csv('/aiffel/data/fraud.csv')

NameError: name 'pd' is not defined

## 데이터 확인하기

해당 데이터셋은 구매 이력에 대한 데이터이다.

trans_date_trans_time: 구매한 날짜와 시간

cc_num: 신용카드 번호
동일한 값이 빈번하게 등장한다. -> 동일한 신용카드를 사용한 모든 트랜잭션들이 한 줄 한 줄 기록되어 있다.

merchant: 구매 상점 이름

category: 구매 상점의 카테고리

amt: 구매 금액

first/last: 이름

gender: 성별

street/city/state/zip/lat/log: 고객의 주소 정보

city_pop: zip에 해당하는 인구 수

job: 고객의 직업

dob(day of birth): 생년월일

trans_num: 트랜잭션별 id (필요X)

unix time: trans_date_trans_time의 날짜와 시간을 unix timestamp 형태로 변환시킨 것 (중복된 컬럼)

merch_lat/merch_long: 상점의 위도/경도

is_fraud: 사기 거래인지 여부


In [ ]:
cc_df

In [ ]:
pd.set_option('display.max_columns', 50)

In [ ]:
cc_df.head()

In [ ]:
cc_df.info()

결측치가 없는 것을 확인할 수 있다.

날짜 관련 컬럼인 trans_date_trans_time과 dob가 object 형태라서,   
해당 컬럼을 사용한다면 datatime 형태로 변환해야 한다.

In [ ]:
cc_df.describe()

amt의 max가 25086으로 크지만, 구매 금액인 걸 감안하면 이상치로 판단할 수는 없다.  
또, fraud건 같은 경우는 많은 금액을 빼돌리기 위해서 큰 금액을 결제하는 경향도 있기 때문에 이상치라도 하더라도 나름대로 의미가 있을 수 있다.  
-> amt 컬럼에 대한 이상치는 별도로 처리하지 않겠다.

is_fraud는 0 아니면 1 값인데, min 값을 보면 0.2%로 아주 적은 숫자만 fraud 케이스인 것을 알 수 있다.  
-> imbalanced data(불균형 데이터)라고 한다.

=> True와 False의 비율이 어느 정도 맞아야 예측 모델을 만들었을 때 좋은 결과를 얻기 쉬운데, 해당 데이터셋처럼 한 쪽으로 치우쳐 있는 경우에는 imbalanced data라고 해서 예측하기가 상당히 까다롭다.

## 불필요한 컬럼 제거하기

In [ ]:
cc_df.head(3)

cc_num(신용카드 번호)는 예측하는 순간에는 필요가 없지만, Feature Engineering을 할 때 동일한 고객(동일한 카드 번호)인지를 확인할 필요가 있다.

merchant는 특정 상점에 fraud 건이 더 빈번하게 발생한다고 생각할 여지가 많지 않다.  
카테고리형 데이터이고, 고윳값들이 너무 많으면 원-핫 인코딩 시 문제가 된다.  
-> drop

first, last의 이름은 필요없는 정보이다.  
-> drop

주소 정보를 담고 있는 lat, long 이외의 나머지 컬럼들은 카테고리 형태이기도 하고 특히 street의 경우에는 매우 다양해서 사실상 의미가 없다.  
-> drop

job도 성별과 마찬가지로 의미가 있을까 생각되지만, 고윳값 개수를 확인해 보면 cc_num과 거의 비슷하다. 즉, 고객 개개인이 다른 직업으로 입력된 것으로 볼 수 있다.  
-> drop

trans_num은 의미가 없는 데이터이다.  
-> drop

unix_time은 trans_date_trans_time과 중복된 정보를 담고 있다.  
-> drop

In [ ]:
cc_df['merchant'].nunique()

In [ ]:
cc_df['job'].nunique()

In [ ]:
cc_df['cc_num'].nunique()

In [ ]:
cc_df.drop(['merchant', 'first', 'last', 'street', 'city', 'state', 'zip', 'job', 'trans_num', 'unix_time'], axis=1, inplace=True)

# Feature Engineering

피처 엔지니어링을 할 때는 데이터 자체에 대한 이해도 굉장히 중요하지만, 목적도 충분히 고려해서 방향을 결정해야 한다.

"fraud 케이스가 언제 발생할까? 이걸 어떻게 찾아낼 수 있을까?"

특정 고객의 평소 패턴과 다른 거래가 발생한다면 해당 케이스는 문제가 있을 것 같다고 의심해 볼 수 있다.

In [ ]:
cc_df.sort_values('cc_num')

## 구매금액의 z-score 계산하기

구매 금액이 평소보다 훨씬 크면 의심해 볼 수 있다.

단순히 얼마 이상이면 detection을 발동시키는 것이 아니라, 해당 고객의 평소 패턴이 중요하다.

In [ ]:
# 해당 cc_num의 모든 거래 건을 추려서 평균과 표준편차를 구한 후,
# 각각의 amt에 대해 연산을 해주면 z-score를 구할 수 있다.

# z-score = (x - 평균) / 표준편차

In [ ]:
temp = pd.DataFrame({'a': [10,20,30,20,10,200], 'b': [100,300,200,150,250,200], 'c': [10,500,20,250,25,200]})

In [ ]:
temp

In [ ]:
temp.mean()

In [ ]:
temp.std()

In [ ]:
(temp['a'] - 48.33) / 74.67

In [ ]:
(temp['b'] - 200) / 70.71

In [ ]:
(temp['c'] - 167.5) / 192.5

이처럼 z-score를 구하면  
평소의 구매 패턴을 기반으로 각각의 구매 금액이 평소보다 얼마나 많이 썼는지/덜 썼는지를 나타내 준다.

'z-scroe가 얼마 이상이면 fraud 건일 것이다' 라고 정의할 필요는 없다. 이는 머신러닝 알고리즘에서 자연스럽게 찾아낼 것이다.

In [ ]:
cc_df['cc_num'].value_counts()

In [ ]:
# 신용카드 번호(cc_num)별 amt의 평균과 표준편차 구하기
# 나중에 DataFrame을 합쳐줘야 하기 때문에 cc_num을 컬럼으로 빼준다.
amt_info = cc_df.groupby('cc_num')['amt'].agg(['mean', 'std']).reset_index()

In [ ]:
amt_info

In [ ]:
cc_df = cc_df.merge(amt_info, on='cc_num', how='left')

In [ ]:
cc_df

In [ ]:
# z-score 구하기
cc_df['amt_z'] = (cc_df['amt'] - cc_df['mean']) / cc_df['std']

In [ ]:
cc_df

아직은 amt_z가 0에서 크게 떨어지지 않은 일반적인 거래 건들만 보이는데,  
is_fraud가 1인 케이스를 확인해 보자.

In [ ]:
cc_df[cc_df['is_fraud'] == 1]

모든 거래 건들이 그런 건 아니지만, z-score가 크게 나온 건들이 꽤 있다.

-> z-score를 구하는 것이 fraud 케이스를 잡아내는 데 꽤 유용하게 쓰일 수 있을 것 같다.

In [ ]:
cc_df.drop(['mean', 'std'], axis=1, inplace=True)

카테고리까지 고려된 z-score도 구해보자.

In [ ]:
# 신용카드 번호(cc_num)별, 카테고리별 amt의 평균과 표준편차 구하기
cat_info = cc_df.groupby(['cc_num', 'category'])['amt'].agg(['mean', 'std']).reset_index()

In [ ]:
cat_info

In [ ]:
# cc_num별로만 합치면, 중복된 cc_num 그리고 다른 카테고리들이 제대로 매칭이 되지 않는다.
cc_df = cc_df.merge(cat_info, on=['cc_num', 'category'], how='left')

In [ ]:
cc_df

In [ ]:
# z-score 구하기
cc_df['cat_amt_z'] = (cc_df['amt'] - cc_df['mean']) / cc_df['std']

In [ ]:
cc_df.drop(['mean', 'std'], axis=1, inplace=True)

In [ ]:
cc_df.head()

구매 금액에 대해서는 평소 행동 패턴에 기반하여 얼마나 이상한지 이상치를 감지해 내는 두 개의 컬럼 amt_z, cat_amt_z을 구했다.

## 결제 시간 관련 feature 분석

고객마다 신용카드를 빈번하게 사용하는 시간대와, 잘 사용하지 않는 시간대가 있을 것이다.

trans_date_trans_time 컬럼에서 시간만 뽑아내서 각 고객이 주로 어느 시간대에 몇 퍼센트의 비중으로 결제를 하는지, 그리고 각각의 거래 건은 그 중에 어디에 속하는지를 하나의 변수로 추가한다.  
-> 해당 결제 건이 평소에 자주 사용하던 시간대의 결제 건인지, 아니면 좀 드문 시간대에 사용한 결제 건인지도 확인할 수 있다.

=> fraud 케이스를 감지할 수 있는 또 하나의 feature가 될 수 있을 것 같다.

In [ ]:
cc_df.info()

In [ ]:
cc_df['hour'] = pd.to_datetime(cc_df['trans_date_trans_time']).dt.hour

In [ ]:
cc_df.head()

신용카드 번호(cc_num)를 기준으로 각 카드가 총 몇 건이 결제가 되었고, 각각 어떤 시간대에 얼마만큼 결제가 되었나를 살펴봐야 한다.

z-score로 구하기에는 제약이 있다.  
예를 들어 23시와 0시는 컴퓨터가 인식하기에는 0과 23만큼의 차이인데, 실제로 시간 단위에서는 한 시간의 차이밖에 안 되는 것이다. 그래서 오히려 같은 시간 범주로 묶어야 되는데 완전히 동떨어진 개념으로 인식하게 된다.

In [ ]:
# 시간을 숫자 그대로 두지 않고, 인위적으로 범위를 정해서 나눠준다.
def hour_func(x):
    if (x >= 6) & (x < 12):
        return 'morning'
    elif (x >= 12) & (x < 18):
        return 'afternoon'
    elif (x >= 18) & (x < 23):
        return 'night'
    else:
        return 'evening'

In [ ]:
cc_df['hour_cat'] = cc_df['hour'].apply(hour_func)

In [ ]:
cc_df['hour_cat'].value_counts()

In [ ]:
# 신용카드 번호(cc_num)별로 총 몇 건의 결제 건이 있는지 구하기
all_cnt = cc_df.groupby('cc_num')['amt'].count().reset_index()

In [ ]:
# 신용카드 번호(cc_num)별, 시간대별로 총 몇 건의 결제 건이 있는지 구하기
hour_cnt = cc_df.groupby(['cc_num', 'hour_cat'])['amt'].count().reset_index()

In [ ]:
all_cnt.head()

In [ ]:
hour_cnt.head()

In [ ]:
# 시간대별 퍼센트를 구하기 위해 hour_cnt에 all_cnt를 붙이기
hour_cnt = hour_cnt.merge(all_cnt, on='cc_num', how='left')

In [ ]:
hour_cnt.head()

In [ ]:
hour_cnt = hour_cnt.rename({'amt_x': 'hour_cnt', 'amt_y': 'total_cnt'}, axis=1)

In [ ]:
hour_cnt.head()

In [ ]:
# total_cnt에 대한 hour_cnt의 퍼센트 구하기
hour_cnt['hour_perc'] = hour_cnt['hour_cnt'] / hour_cnt['total_cnt']

In [ ]:
hour_cnt.head(10)

In [ ]:
# 동일한 cc_num에서 hour_perc의 합이 1이 되는지 확인
hour_cnt.loc[0:3]['hour_perc'].sum()

In [ ]:
# 동일한 cc_num에서 hour_perc의 합이 1이 되는지 확인
hour_cnt.groupby('cc_num')['hour_perc'].sum()

In [ ]:
# hour_perc 컬럼을 원래 데이터프레임 cc_df에 붙이기 위한 컬럼 정리
hour_cnt = hour_cnt[['cc_num', 'hour_cat', 'hour_perc']]

In [ ]:
# cc_df에 hour_cnt를 합치기
cc_df = cc_df.merge(hour_cnt, on=['cc_num', 'hour_cat'], how='left')

In [ ]:
cc_df.head()

In [ ]:
# 불필요한 컬럼 제거하기
cc_df.drop(['trans_date_trans_time', 'hour', 'hour_cat'], axis=1, inplace=True)

시간에 대한 정보들을 얻었다.

이외에도,
요일을 찾아서 주말과 평일을 구분해 본다든가

이전 거래와의 시간 혹은 날짜 차이를 구해 본다든가  
-> 카드를 쓰다가 다른 카드르 발급받아서 한동안 사용하지 않을 수 있는데, 만약 세 달 동안 안쓰던 카드가 갑자기 높은 금액으로 결제됐다고 한다면, 이 케이스도 fraud detection해 볼 수 있는 부분이다.

## 거리 관련 feature 분석

## 나이 feature 만들어보기

## 범주형 데이터의 One-Hot Encoding